# OpenVINO 

## Установим Зависимости

In [ ]:
#!pip install -U openvino nncf
# # или пре-релизная версия:
#!pip install --pre -U openvino --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly git+https://github.com/openvinotoolkit/nncf.git

#!pip install transformers[torch] datasets evaluate ipywidgets

## Скачиваем Предобученную Модель

Берем классификационную модель из [Huggingface Hub](https://huggingface.co/models?pipeline_tag=text-classification&sort=trending&search=sst2)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_id = "philschmid/MiniLM-L6-H384-uncased-sst2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_model = AutoModelForSequenceClassification.from_pretrained(model_id)

hf_model.eval()

Сохраним оригинальные веса

In [2]:
import torch
torch.save(hf_model.state_dict(), "model_sst.pth")

## Конвертируем Модель в OpenVINO


In [ ]:
import openvino as ov

hf_model.eval()
inputs = {**tokenizer("This lesson was amazing!", 
                      return_tensors="pt")}
ov_model = ov.convert_model(hf_model, example_input=v)
ov.save_model(ov_model, 'model.xml')

compiled_model = ov.compile_model(ov_model)
print(compiled_model(inputs))

torch.Size([1, 7])
torch.Size([1, 7])


In [ ]:
inputs

{'input_ids': tensor([[  101,  2023, 10800,  2001,  6429,   999,   102]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

Конвертируем модель с простым входным тензором

Провалидируем предсказания сконвертированной модели:

- PyTorch accuracy:  **0.90138**
- OpenVINO accuracy: **0.90138**

In [51]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [52]:
import torch
import numpy as np
from datasets import load_dataset
import evaluate


val_dataset = load_dataset("glue", "sst2", split="validation")
accuracy = evaluate.load("accuracy")


@torch.no_grad
def accuracy_evaluate(model, dataset=val_dataset, accuracy=accuracy):   
    for sample in dataset:
        tokenized = {**tokenizer(sample["sentence"], return_tensors="pt")}
        logits = model(tokenized)["logits"]
        pred = np.argmax(logits, axis=1)
        accuracy.add(references=sample["label"], predictions=pred)

    return accuracy.compute()["accuracy"]


print(f"PyTorch accuracy:  {accuracy_evaluate(lambda x: hf_model(**x))}")
print(f"OpenVINO accuracy: {accuracy_evaluate(compiled_model)}")

PyTorch accuracy:  0.9013761467889908
OpenVINO accuracy: 0.9013761467889908


## Benchmark

Добавим несколько инференсов в бенчмарк для того, чтобы получить более точные результаты.

* Pytorch:   7.33648s, FPS=118.858, latency: 0.00647s, 0.00830s, 0.01716s
* Openvino:  5.27496s, FPS=165.309, latency: 0.00343s, 0.00606s, 0.01561s

In [ ]:
from time import perf_counter
from statistics import median


@torch.no_grad
def benchmark(model, dataset):
    tokenized_dataset = [{**tokenizer(sample["sentence"], return_tensors="pt")} for sample in dataset]

    # warmup
    for data in tokenized_dataset[:10]:
        model(data)
        
    times = []
    for _ in range(3):
        for data in tokenized_dataset:
            start = perf_counter()
            model(data)
            end = perf_counter()
            times.append(end - start)

    return (
        f"{sum(times):.5f}s, FPS={(len(dataset) / sum(times)):.3f}, "
        f"latency: {min(times):.5f}s, {median(times):.5f}s, {max(times):.5f}s"
    )

print("Pytorch:  ", benchmark(lambda x: hf_model(**x), val_dataset))
print("Openvino: ", benchmark(lambda x: compiled_model(x), val_dataset))

Pytorch:   20.02967s, FPS=43.535, latency: 0.00604s, 0.00750s, 0.01791s
Openvino:  15.10048s, FPS=57.747, latency: 0.00312s, 0.00576s, 0.01241s


## Inference Hints

Скомпилируем модель с разными инференс хинтами и сравним результаты бенчмарка.

In [9]:
import openvino.properties as props
import openvino.properties.hint as hints

core = ov.Core()
print(core.available_devices)
print(core.get_property('CPU', props.device.capabilities))

['CPU', 'GPU']
['FP32', 'INT8', 'BIN', 'EXPORT_IMPORT']


In [ ]:
compiled_througput = ov.compile_model(
    ov_model, 
    "CPU", 
    {hints.performance_mode: hints.PerformanceMode.THROUGHPUT}
)
print("Openvino: ", benchmark(lambda x: compiled_througput(x), val_dataset))

In [ ]:
ov_model = ov.convert_model(hf_model, example_input= dict(inputs))

compiled_througput = ov.compile_model(
    ov_model, 
    "CPU", 
    {hints.performance_mode: hints.PerformanceMode.THROUGHPUT}
)
print("Openvino: ", benchmark(lambda x: compiled_througput(x), val_dataset))

Openvino:  10.86417s, FPS=80.264, latency: 0.00192s, 0.00393s, 0.01735s


In [ ]:
compiled_latency = ov.compile_model(
    ov_model, 
    "CPU", 
    {hints.performance_mode: hints.PerformanceMode.LATENCY}
)
print("Openvino: ", benchmark(lambda x: compiled_latency(x), val_dataset))

Openvino:  11.55482s, FPS=75.466, latency: 0.00233s, 0.00439s, 0.00849s


## Async Inference

Перепишем  бенчмарк под асинхронный инференс. Он должен принимать на вход асинхронную очередь и датасет.

### Простой Бенчмарк
Простая версия бенчмарка должна замерить FPS:

In [ ]:
from typing import Dict, Any, Optional


def completion_callback(
    infer_request: ov.InferRequest, user_data: Optional[Dict[str, Any]] = None
) -> None:
    ...

infer_queue = ov.AsyncInferQueue(compiled_througput)
infer_queue.set_callback(completion_callback)

def simple_benchmark_async(queue, dataset):
    tokenized_dataset = [{**tokenizer(sample["sentence"], return_tensors="np")} for sample in dataset]

    # warmup
    for data in tokenized_dataset[:10]:
        queue.start_async(data)
    queue.wait_all()

    elapsed_start = perf_counter()
    for idx, data in enumerate(tokenized_dataset):
        queue.start_async(data) 
    queue.wait_all()
    elapsed_end = perf_counter()
    elapsed = elapsed_end - elapsed_start

    return f"{elapsed:.5f}s, FPS={(len(dataset) / elapsed):.3f}"

print("Openvino: ", simple_benchmark_async(infer_queue, val_dataset))

Openvino:  1.98564s, FPS=439.152


### Добавим Измерение latency В Асинхронный Бенчмарк

Используем `completion_callback` для подсчёта latency.

In [ ]:
def completion_callback(
    infer_request: ov.InferRequest,
    user_data: Dict[str, Any],
) -> None:
    end = perf_counter()  # инференс завершился, заменяем время
    idx = user_data["idx"]
    times = user_data["times"]
    times[idx] = end - times[idx]  # вычитаем время начала 


# используем существующую очередь, переназначим коллбэк
infer_queue.set_callback(completion_callback)


def benchmark_async(queue, dataset):
    tokenized_dataset = [{**tokenizer(sample["sentence"], return_tensors="np")} for sample in dataset]
    times = [0 for _ in range(len(dataset))]

    # warmup
    for data in tokenized_dataset[:10]:
        queue.start_async(data, {"idx": 0, "times": times})
    queue.wait_all()
    
    start = perf_counter()
    for idx, data in enumerate(tokenized_dataset):
        # записываем время старта реквеста в массив по индексу входных данных
        times[idx] = perf_counter()
        # передаём индекс и массив с началами вместе с входными данными
        queue.start_async(data, {"idx": idx, "times": times})
    # ждём пока завершатся все реквесты
    queue.wait_all()

    # замеряем время конца инференса
    end = perf_counter()
    # для общего времени исполнения уже нельзя брать sum(times), так как реквесты исполняются одновременно
    elapsed = end - start
    
    return (
        f"{elapsed:.5f}s, FPS={(len(dataset) / elapsed):.3f}, "
        f"latency: {min(times):.5f}s, {median(times):.5f}s, {max(times):.5f}s"
    )


print("Openvino: ", benchmark_async(infer_queue, val_dataset))

Openvino:  2.00049s, FPS=435.893, latency: 0.00000s, 0.00143s, 0.01041s


`InferRequest` объект сам замеряет latency во время инференса, поэтому можно просто достать время оттуда. Время замеряется в плюсах, поэтому latency получается немного меньше. benchmark app для замеров latency тоже берёт информацию из реквеста.

In [ ]:
def latency_from_ir_completion_callback(
    infer_request: ov.InferRequest,
    user_data: Dict[str, Any],
) -> None:
    times = user_data["times"]
    idx = user_data["idx"]
    times[idx] = infer_request.latency * 1e-3  # ms -> s


infer_queue.set_callback(latency_from_ir_completion_callback)
print("Openvino: ", benchmark_async(infer_queue, val_dataset))

### Async Accuracy Evaluation

Напишим функцию, которая измеряет `accuracy` в режиме асинхронного инференса. Для получения доступа к данным из `InferRequest` можно использовать метод `infer_request.get_tensor("<output_name>").data`. Синхронная функция для референса:
```python
def accuracy_evaluate(model, dataset=val_dataset, accuracy=accuracy):   
    for sample in dataset:
        tokenized = {**tokenizer(sample["sentence"], return_tensors="pt")}
        logits = model(tokenized)["logits"]
        pred = np.argmax(logits, axis=1)
        accuracy.add(references=sample["label"], predictions=pred)

    return accuracy.compute()["accuracy"]
```

In [16]:
from tqdm import tqdm

predictions = []
references = []

def completion_callback(
    infer_request: ov.InferRequest,
    user_data: Dict[str, Any],
) -> None:
    logits = infer_request.get_tensor("logits").data
    pred = np.argmax(logits, axis=1)

    predictions.append(pred.item())
    references.append(user_data["label"])


# используем существующую очередь, переназначим коллбэк
infer_queue.set_callback(completion_callback)


def async_accuracy_evaluate(queue, dataset=val_dataset, accuracy=accuracy):

    
    for sample in tqdm(dataset):
        tokenized = {**tokenizer(sample["sentence"], return_tensors="np")}
        user_data = {"label": sample["label"]}
        queue.start_async(tokenized, user_data) 


    queue.wait_all()
    accuracy.add_batch(references=references, predictions=predictions)
    return accuracy.compute()["accuracy"]


print(f"Model accuracy: {async_accuracy_evaluate(infer_queue)}")

100%|██████████| 872/872 [00:02<00:00, 434.32it/s]

Model accuracy: 0.9013761467889908


## Benchmark App

### Измерим Производительность Модели с Помощью CLI benchmark_app

Чтобы не ждать по минуте используем флаг `-t 30`

In [ ]:
!benchmark_app -m "bert.xml" -shape [1,128] -t 30 | tail

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2025.0.0-17942-1f68be9f594-releases/2025/0
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2025.0.0-17942-1f68be9f594-releases/2025/0
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to PerformanceMode.THROUGHPUT.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 43.67 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : i64 / [...] / [?,?]
[ INFO ]     attention_mask , 63 (node: attention_mask) : i64 / [...] / [?,?]
[ INFO ]     token_type_ids (node: token_type_ids) : i64 / [...] / [?,?]
[ INFO ] Model outputs:
[ INFO ]     logits (node:

###  Попробуем подобрать параметры, чтобы увеличить FPS относительно

101 FPS -> 156 FPS

In [ ]:
!mkdir -p benchmark_report
!benchmark_app -hint throughput -m model.xml -t 30 -shape [1,128] -t 30 -report_folder benchmark_report -pc -pcsort simple_sort \
-report_type average_counters | tail

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2025.0.0-17942-1f68be9f594-releases/2025/0
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2025.0.0-17942-1f68be9f594-releases/2025/0
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 18.68 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : i64 / [...] / [?,?]
[ INFO ]     attention_mask , 63 (node: attention_mask) : i64 / [...] / [?,?]
[ INFO ]     token_type_ids (node: token_type_ids) : i64 / [...] / [?,?]
[ INFO ] Model outputs:
[ INFO ]     logits (node: __module.classifier/aten::linear/Add) : f32 / [...] / [?,2]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch s

In [ ]:
!head -n 3 benchmark_report/benchmark_sorted_report.csv

> ⚠️ Окончательный замер всегда нужно проводить с отключенными perf_counter'ами, так как сбор такой статистики замедляет инференс


In [ ]:
import json


with open("ov_config.json", "w") as config_file:
    json.dump(
        {
            "CPU": {
                "NUM_STREAMS": 24,
                "INFERENCE_NUM_THREADS": 48,
            }
        },
        config_file, 
    )

!benchmark_app -m "bert.xml" -shape [1,128] -d CPU -load_config ov_config.json

За счёт ухудшения latency удалось немного повысить throughput.

> ⚠️ Такие измерения нужно производить непосредственно на железе, которое будет использоваться для инференса 

## NNCF

### Дефолтная Квантизация

Квантизуем модель с дефолтными параметрами. Замерим accuracy.

In [38]:
ov_model = ov.convert_model(hf_model, example_input= dict(inputs))

config = {hints.performance_mode: hints.PerformanceMode.THROUGHPUT}
compiled_througput = ov.compile_model(ov_model, "CPU", config)
print("Openvino: ", benchmark(lambda x: compiled_througput(x), val_dataset))

Openvino:  11.09594s, FPS=78.587, latency: 0.00203s, 0.00402s, 0.01728s


In [ ]:
import nncf


def transform_fn(text):
    return {**tokenizer(text["sentence"], return_tensors="np")}


# возьмём для калибрации другой датасет
test_dataset = load_dataset("glue", "sst2", split="test[:300]")
calibration_dataset = nncf.Dataset(test_dataset, transform_fn)
quntized_model = nncf.quantize(
    ov_model, 
    calibration_dataset=calibration_dataset,
    preset=nncf.QuantizationPreset.MIXED,
    target_device=nncf.TargetDevice.CPU,  # важно
    model_type=nncf.ModelType.TRANSFORMER,  # очень важно!
)
ov.save_model(quntized_model, "qbert.xml", compress_to_fp16=False)

In [56]:
compiled_quantized = ov.compile_model(quntized_model)
print(f"OpenVINO accuracy: {accuracy_evaluate(compiled_quantized)}")

OpenVINO accuracy: 0.8474770642201835


Против оригинальной модели

In [54]:
print(f"OpenVINO accuracy: {accuracy_evaluate(compiled_througput)}")

OpenVINO accuracy: 0.9013761467889908


Замерим FPS квантизованной модели с помощью benchmark функции или benchmark_app:

156 FPS -> 401 FPS

In [ ]:
!benchmark_app -m "qbert.xml" -shape [1,128] -t 30 | tail

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2025.0.0-17942-1f68be9f594-releases/2025/0
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2025.0.0-17942-1f68be9f594-releases/2025/0
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 18.73 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : i64 / [...] / [?,?]
[ INFO ]     attention_mask , 63 (node: attention_mask) : i64 / [...] / [?,?]
[ INFO ]     token_type_ids (node: token_type_ids) : i64 / [...] / [?,?]
[ INFO ] Model outputs:
[ INFO ]     logits (node: __module.classifier/aten::linear/Add) : f32 / [...] / [?,2]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch s

### Accuracy Control

Квантизуем модель так, чтобы потеря accuracy была в пределах 1%.

In [ ]:
# разделим валидационный датасет для финальной валидации
validation_dataset = list(val_dataset)
final_test_dataset, validation_dataset = validation_dataset[:-300], validation_dataset[-300:]
validation_dataset = nncf.Dataset(validation_dataset, transform_fn)

quntized_model = nncf.quantize_with_accuracy_control(
    model=ov_model,
    calibration_dataset=calibration_dataset,
    preset=nncf.QuantizationPreset.PERFORMANCE,
    validation_dataset=validation_dataset,
    validation_fn=accuracy_evaluate,  # функция замера accuracy переиспользуется
    max_drop=0.01,
    target_device=nncf.TargetDevice.CPU,
    drop_type=nncf.DropType.ABSOLUTE,
    # уберём тип модели, чтобы получить accuracy drop больше 1%
    # иначе квантизация будет совпадать с дефолтной 🤷
    # model_type=nncf.ModelType.TRANSFORMER, 
)
ov.save_model(quntized_model, "qbert_acc.xml", compress_to_fp16=False)

In [ ]:
compiled_quantized_acc = ov.compile_model(
    "qbert_acc.xml", "CPU", {hints.performance_mode: hints.PerformanceMode.THROUGHPUT}
)

print(f"Openvino: {accuracy_evaluate(compiled_througput, final_test_dataset)}")
print(f"Quantized Openvino :  {accuracy_evaluate(compiled_quantized, final_test_dataset)}")
print(f"Openvino quantized with acc:  {accuracy_evaluate(compiled_quantized_acc, final_test_dataset)}")

In [ ]:
from openvino_tokenizers import convert_tokenizer, connect_models


def get_connected_model(hf_model, tokenizer):
    example_input = {**tokenizer("test", return_tensors="pt")}
    ov_model = ov.convert_model(hf_model, example_input=example_input)
    ov_tokenizer = convert_tokenizer(tokenizer)
    return connect_models(ov_tokenizer, ov_model)


get_connected_model(hf_model, tokenizer)

Как видно из результатов на тестовом датасете:
- Определённый дроп на валидационном датасете ничего не гарантирует.
- Указать тип модели бывает важнее, чем дать валидационный датасет.

In [69]:
ov.save_model(quntized_model_best, "qbert_best.xml", compress_to_fp16 = False)

compiled_quantized_best = ov.compile_model(quntized_model_best)
print(f"OpenVINO accuracy: {accuracy_evaluate(compiled_quantized_best)}")

OpenVINO accuracy: 0.8956422018348624


156 FPS -> 285.53 FPS

In [70]:
!benchmark_app -hint throughput -m qbert_best.xml -t 30 -shape "input_ids[1,128],attention_mask[1,128],token_type_ids[1,128]"

[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2025.0.0-17942-1f68be9f594-releases/2025/0
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2025.0.0-17942-1f68be9f594-releases/2025/0
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 17.17 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : i64 / [...] / [?,?]
[ INFO ]     attention_mask , 63 (node: attention_mask) : i64 / [...] / [?,?]
[ INFO ]     token_type_ids (node: token_type_ids) : i64 / [...] / [?,?]
[ INFO ] Model outputs:
[ INFO ]     logits (node: __module.classifier/aten::linear/Add) : f32 / [...] / [?,2]
[Step 5/11] Resizing model to match image sizes and given batch
[ INFO ] Model batch s

# Дополнительно

Добавим в модель:
1. Токенизационный препроцессинг с помощью `openvino-tokenizers`
2. Постпроцессинг в модель, чтобы она сразу отдавала результат `np.argmax(logits, axis=1)`

In [ ]:
# !pip install --pre -U openvino-tokenizers --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly git+https://github.com/openvinotoolkit/nncf.git

In [ ]:
from openvino_tokenizers import convert_tokenizer, connect_models


def get_connected_model(hf_model, tokenizer):
    example_input = {**tokenizer("test", return_tensors="pt")}
    ov_model = ov.convert_model(hf_model, example_input=example_input)
    ov_tokenizer = convert_tokenizer(tokenizer)
    return connect_models(ov_tokenizer, ov_model)


get_connected_model(hf_model, tokenizer)